In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib
#matplotlib.rcParams.update({'font.size': 12,'font.family':'serif','font.serif':['Computer Modern']})

import ipywidgets as widgets
import datetime
import pandas as pd
import os
import hypermri
import pydicom

from scipy.optimize import curve_fit
import scipy.io as sio
from matplotlib import cm

import hypermri.utils.utils_general as utg
import hypermri.utils.utils_spectroscopy as uts
import hypermri.utils.utils_fitting as utf

from hypermri.utils.utils_fitting import def_fit_params, fit_data_pseudo_inv
from hypermri.utils.utils_general import get_gmr
from hypermri.utils.utils_spectroscopy import get_metab_cs_ppm, make_NDspec_6Dspec


import sys
# define paths:
sys.path.append('../../')
import Template_utsw
basepath,savepath = Template_utsw.import_all_packages()

# Autoreload extension so that you dont have to reload the kernel every time something is changed in the hypermri package
%load_ext autoreload
%autoreload 2

%matplotlib widget
def get_colors_from_cmap(cmap_name, N):
    cmap = plt.get_cmap(cmap_name)
    colors = cmap(np.linspace(0, 1, N))
    return colors

In [ ]:
revision_path=r'/Publication/Revision1/RefittedData/SP_Brain_healthy/'

In [ ]:
study_ids=['3TC230','3TC718','3TC741','3TC755']
injections=[[1,],[1,2],[1,2],[1,2]]

In [ ]:
all_data = pd.DataFrame()
c=0
for idx,study_id in enumerate(study_ids):
    for injection in injections[idx]:
        data=pd.read_excel(revision_path+str(study_id)+'_results_rawdata_inj_'+str(injection)+'_thld_snr_fit.xlsx')
        data['Unnamed: 0'] = np.ones((4,1),dtype=int)*c
        data = data.rename(columns={'Unnamed: 0':'ScanNum'})
        c+=1
        all_data = pd.concat([all_data,data])
all_data=all_data.reset_index(drop=True)



In [ ]:
plt.close('all')
fig,ax=plt.subplots(1,figsize=(8,3),tight_layout=True)
colors=get_colors_from_cmap('tab10',10)

h=0
for id_value in all_data['ScanNum'].unique():
    subset = all_data[all_data['ScanNum'] == id_value]
    c=0
    for slic in subset['slice']:
        ax.errorbar(x=id_value+h+c,y=subset['T'].iloc[slic],yerr=subset['dT'].iloc[slic],capsize=3,capthick=1,fmt='o',color=colors[slic])
        c+=0.5
    h+=4
ax.set_xlabel('dataset')
ax.set_ylabel('T [°C]')

In [ ]:
y_values=[]
ids=[]
y_err_values=[]
for ID in all_data['ScanNum'].unique():
    subset = all_data[all_data['ScanNum'] == ID]
    y_values.append(np.array(subset['T']))
    y_err_values.append(np.array(subset['dT']))
    ids.append(subset['ID'].iloc[0])
y_values=np.array(y_values)
y_err_values=np.array(y_err_values)


# Sample data
x_labels = ids


# Parameters
num_x_labels = len(x_labels)
num_values_per_label = len(y_values[0])
bar_width = 0.3  # Width of each bar
group_spacing=3
x = np.arange(0, num_x_labels * group_spacing, group_spacing)
# Create the plot
fig, ax = plt.subplots()

for i in range(num_values_per_label):
    ax.errorbar(x + i * bar_width, [y[i] for y in y_values], capsize=5,fmt='o',yerr=[y[i] for y in y_err_values], label=f'slice {i + 1}')

# Set x-ticks and labels
ax.set_xticks(x + bar_width * (num_values_per_label - 1) / 2)
ax.set_xticklabels(x_labels)

# Add labels and title (optional)
ax.set_xlabel('Dataset')
ax.set_ylabel('T [°C]')
ax.set_title('Temperature per slice for all datasets')
ax.legend()

# Show the plot
plt.show()
plt.savefig(savepath+'utsw_all_slices_plot_snr_thld.png',dpi=300)

## Save data for final loading

In [ ]:
all_data.to_excel(revision_path+'UTSW_healthy_volunteers_final_snr_thld.xlsx')